In [1]:
from __future__ import print_function

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import tree

from matplotlib import pyplot
import numpy as np
import random
%matplotlib inline
pd.options.mode.chained_assignment = None
bank = pd.read_csv('bank-full.csv',sep=";")


In [2]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
bank = bank.sort_values(by=['y'])
no_data_cnt = len(bank[bank['y']=='no'])
print(no_data_cnt)
bank = bank.iloc[:80000, :]

39922


In [4]:
m = bank.shape[1]
X = bank.iloc[:,0:(m-1)]
y = bank.iloc[:,(m-1):]

y    nonononononononononononononononononononononono...
dtype: object


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [6]:
X_train.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
dtype: object

In [7]:
X_train_onehot = pd.get_dummies(X_train)
X_test_onehot = pd.get_dummies(X_test)
y_train.loc[:,'y'] = y_train.loc[:,'y'].map({'no':0,'yes':1})
y_test.loc[:,'y'] = y_test.loc[:,'y'].map({'no':0,'yes':1})

In [9]:
print(np.sum(y_train), np.sum(y_test))
print(np.mean(y_train), np.mean(y_test))


y    3968
dtype: int64 y    1321
dtype: int64
y    0.117023
dtype: float64 y    0.116872
dtype: float64


In [6]:
X_train_onehot

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
38015,30,303,14,176,2,365,4,0,0,1,...,0,0,1,0,0,0,1,0,0,0
44341,29,0,29,99,6,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24269,49,7,17,50,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4680,38,273,20,314,2,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6450,45,-173,27,315,2,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
15252,27,24,17,320,2,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28981,50,1088,2,749,1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41779,74,1124,13,200,2,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
11037,39,1551,17,264,16,-1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
18916,53,0,4,134,2,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
T=100 #Max number of trees

m = X_train_onehot.shape[0]
N = X_train_onehot.shape[1]
feature_portion = 0.5
W = [1.0/m for i in range(0,m,1)]
W = pd.DataFrame({'Weight':list(W)},index = X_train_onehot.index)
feature_record = pd.DataFrame()

# List of trees
clfs = []
# List of weights of trees
alphas = []

for i in range(1,T+1):
    # Prepare training sample subset (bagging)
    X_train_onehot_sample, X_test_onehot_sample, y_train_sample, y_test_sample = train_test_split(X_train_onehot, y_train, test_size=0.20, random_state=33)
    selected_features =  [random.randint(0,N-1) for j in range(0,int(round(N*feature_portion)))]
    feature_record=feature_record.append(pd.DataFrame([selected_features]), ignore_index=True)    
    X_train_onehot_sample = X_train_onehot_sample.iloc[:,selected_features]
    X_test_onehot_sample = X_test_onehot_sample.iloc[:,selected_features]
    # Prepare tree classifier
    clf = tree.DecisionTreeClassifier(criterion='entropy',max_depth=None)
    # Weight of current training sample
    w_ = W.loc[X_train_onehot_sample.index,"Weight"].tolist()
    
    # Train decision tree
    clf.fit(X=X_train_onehot_sample, y=y_train_sample,sample_weight=w_)
    # Make prediction
    pred = clf.predict(X_train_onehot.iloc[:,selected_features])
    
    # Calculate weighted error rate of current tree
    print("sum(np.array(W)): {}".format(sum(np.array(W))))
    eps = sum(np.array(W)[(np.ravel(pred) != np.ravel(y_train))]) / sum(np.array(W))
    print("eps: ", eps)
    if eps < 1e-20 :
        print("eps == {}. Break".format(eps))
        break
    # Compute weight of decision tree
    alpha = (1/2)*np.log( (m-1)*(1-eps)/eps )
    print("Alpha:", alpha)
    alphas.append(alpha)
    
    
#   print( np.array([ int(y==p) for y,p in zip(np.ravel(y_train), pred) ])[np.where(W>0)]         )
    print("if all focused preditcted correct then zero:",sum( (np.ravel(pred) != np.ravel(y_train))[np.ravel(np.array(W)>0) & np.ravel(~np.isnan(W))]) ,sep="")
    
    # Update weight of training sample
    if alpha > 0 :
        exp_alphas = [ np.exp(-alpha) if y==p else np.exp(alpha) for y,p in zip(np.ravel(y_train), pred) ]
        W = np.multiply(W, exp_alphas) / np.sum(np.multiply(W, exp_alphas))

        
        # Save trained tree to list 
        clfs.append(clf)
    # If alpha < 0, reject the tree
        print('in if then loop')
    else :
        print("Tree {} is rejected.".format(i))


        

    
    #X.iloc[X_train.index,:]
    print('The accuracy of Tree ',str(i), ' is ',round(clf.score(X=X_test_onehot_sample, y=y_test_sample),5),sep='')
    


sum(np.array(W)): [1.]
eps:  [0.02913767]
Alpha: [6.96876486]
if all focused preditcted correct then zero:988
in if then loop
The accuracy of Tree 1 is 0.8555
sum(np.array(W)): [1.]
eps:  [0.45140453]
Alpha: [5.31318708]
if all focused preditcted correct then zero:1387
in if then loop
The accuracy of Tree 2 is 0.82734
sum(np.array(W)): [1.]
eps:  [0.64796986]
Alpha: [4.91062469]
if all focused preditcted correct then zero:5900
in if then loop
The accuracy of Tree 3 is 0.77809
sum(np.array(W)): [1.]
eps:  [0.66435273]
Alpha: [4.87431217]
if all focused preditcted correct then zero:1010
in if then loop
The accuracy of Tree 4 is 0.85196
sum(np.array(W)): [1.]
eps:  [0.81249105]
Alpha: [4.48254922]
if all focused preditcted correct then zero:1634
in if then loop
The accuracy of Tree 5 is 0.81996
sum(np.array(W)): [1.]
eps:  [0.74998871]
Alpha: [4.66641234]
if all focused preditcted correct then zero:1025
in if then loop
The accuracy of Tree 6 is 0.85137
sum(np.array(W)): [1.]
eps:  [0.7008

sum(np.array(W)): [1.]
eps:  [0.99788305]
Alpha: [2.13785884]
if all focused preditcted correct then zero:3240
in if then loop
The accuracy of Tree 52 is 0.81864
sum(np.array(W)): [1.]
eps:  [0.48489116]
Alpha: [5.24591525]
if all focused preditcted correct then zero:6952
in if then loop
The accuracy of Tree 53 is 0.71837
sum(np.array(W)): [1.]
eps:  [0.99987272]
Alpha: [0.73119259]
if all focused preditcted correct then zero:2891
in if then loop
The accuracy of Tree 54 is 0.8325
sum(np.array(W)): [1.]
eps:  [0.99990487]
Alpha: [0.58558411]
if all focused preditcted correct then zero:11372
in if then loop
The accuracy of Tree 55 is 0.60926
sum(np.array(W)): [1.]
eps:  [0.99997134]
Alpha: [-0.01437689]
if all focused preditcted correct then zero:4681
Tree 56 is rejected.
The accuracy of Tree 56 is 0.76069
sum(np.array(W)): [1.]
eps:  [0.00014792]
Alpha: [9.62504779]
if all focused preditcted correct then zero:15787
in if then loop
The accuracy of Tree 57 is 0.50133
sum(np.array(W)): [1.

In [8]:
np.sum(np.array(pred)!= np.array(y_train),axis=0)

array([ 3979, 29929,  3979, ...,  3979,  3979,  3979])

In [9]:
print(np.array(y_train)[1].shape)
print(np.array(pred).shape)

(1,)
(33908,)


In [18]:
alphas /= sum(alphas)


ensemble_prob = np.zeros(len(X_train_onehot))
for i,clf in enumerate(clfs) :
    prob = clf.predict_proba(X_train_onehot.iloc[:,list(feature_record.iloc[i,:])])
    ensemble_prob += prob[:,1]*alphas[i]
    #print(ensemble_prob)

#ensemble_prob /= len(clfs)
#ensemble_pred = ensemble_prob
#ensemble_pred[ensemble_pred>=0.5]=1
#ensemble_pred[ensemble_pred<0.5]=0
#print(sum(ensemble_pred))
#print(ensemble_prob)
#ensemble_pred = [ int(pb>0.5) for pb in ensemble_prob ]
#print(len(ensemble_pred))
ensemble_pred= ensemble_prob
ensemble_pred[ensemble_prob>=0.5]=1
ensemble_pred[ensemble_prob<0.5]=0
sum(ensemble_pred)

0.0

In [26]:
ensemble_prob>0

array([False, False, False, ..., False, False, False])

In [11]:
np.mean((np.array(y_train)==np.array(ensemble_pred))[0])

0.9939542290904801

In [27]:
alphas /= sum(alphas)


ensemble_prob = np.zeros(len(X_test_onehot))
for i,clf in enumerate(clfs) :
    prob = clf.predict_proba(X_test_onehot.iloc[:,list(feature_record.iloc[i,:])])
    ensemble_prob += prob[:,1]*alphas[i]
    #print(ensemble_prob)

#ensemble_prob /= len(clfs)
#ensemble_pred = ensemble_prob
#ensemble_pred[ensemble_pred>=0.5]=1
#ensemble_pred[ensemble_pred<0.5]=0
#print(sum(ensemble_pred))
#print(ensemble_prob)
#ensemble_pred = [ int(pb>0.5) for pb in ensemble_prob ]
#print(len(ensemble_pred))
ensemble_pred= ensemble_prob
ensemble_pred[ensemble_prob>=0.5]=1
ensemble_pred[ensemble_prob<0.5]=0
sum(ensemble_pred)

0.0

In [28]:
test1=((np.ravel(y_test)==np.ravel(ensemble_pred)))
np.mean(test1)

0.8841015659559409

In [31]:
1 - np.mean(y_test)

y    0.884102
dtype: float64

In [14]:
pred = np.array(ensemble_pred)
corr = [ int(y==p) for y,p in zip(pred, np.array(y_test)) ]
print( np.mean(corr) )

0.8841015659559409


In [15]:
feature_record

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,39,3,17,33,46,34,24,46,0,49,...,23,43,19,37,36,12,32,17,28,5
1,27,3,17,9,41,25,30,1,6,44,...,9,17,37,45,23,2,45,4,6,19
2,14,14,49,40,15,33,13,28,8,11,...,46,4,45,9,8,40,30,32,22,18
3,32,0,36,16,26,18,32,42,49,18,...,43,36,40,29,34,5,17,0,8,5
4,29,2,41,25,19,32,4,44,30,49,...,48,35,13,10,48,5,42,22,26,41
5,8,44,30,31,1,46,37,19,15,43,...,40,24,24,48,5,31,43,31,3,28
6,29,40,36,42,16,21,5,18,40,29,...,6,49,26,50,4,30,30,12,9,47
7,12,3,2,0,16,45,47,10,46,44,...,25,26,38,32,16,41,26,25,48,45
8,14,27,24,26,8,50,28,33,36,35,...,28,4,42,35,30,36,16,44,39,14
9,2,45,19,50,2,22,16,33,12,9,...,7,18,14,40,44,33,26,39,48,31


In [16]:
importances = clf.feature_importances_ 
importance_frame = pd.DataFrame({'Importance': list(importances), 'Feature': list(X_train_onehot.columns)})
importance_frame.sort_values(by = 'Importance', inplace = True)
importance_frame.plot(kind = 'barh', x = 'Feature', figsize = (16,100), color = 'orange')

ValueError: arrays must all be same length

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train_onehot, y_train)
print('The accuracy of Random Forest Classifier on testing set:', rfc.score(X_test_onehot, y_test))
np.mean(np.array(rfc.predict(X_test_onehot))==np.array(y_test))

In [ ]:
pred = np.array(rfc.predict(X_test_onehot))
corr = [ int(y==p) for y,p in zip(pred, np.array(y_test)) ]
print( np.mean(corr) )

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import DMatrix
xgbc = XGBClassifier()
xgbc.fit(X_train_onehot, y_train)
print('The accuracy of eXtreme Gradient Boosting Classifier on testing set:', xgbc.score(X_test_onehot, y_test))

xgdmat = DMatrix(X_train_onehot, y_train) 

In [ ]:
from sklearn.model_selection import GridSearchCV
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1) 
# Optimize for accuracy since that is the metric used in the Adult Data Set notation

optimized_GBM.fit(X_train_onehot, y_train)

In [ ]:
optimized_GBM.best_estimator_

In [ ]:
optimized_GBM2 = GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=3, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'subsample': [0.7, 0.8, 0.9], 'learning_rate': [0.1, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

optimized_GBM2.fit(X_train_onehot, y_train)

In [ ]:
optimized_GBM2.best_estimator_

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':3} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping that minimizes error

cv_xgb.tail(5)

In [ ]:
from xgboost import plot_tree
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns
sns.set(font_scale = 1.5)

plot_tree(xgbc,rankdir='LR')
fig = pyplot.gcf()
fig.set_size_inches(150, 100)
#fig.savefig('tree.png')

In [ ]:
from xgboost import plot_importance
plot_importance(xgbc)
fig = pyplot.gcf()
fig.set_size_inches(150, 100)

In [ ]:
importances = xgbc.feature_importances_
importance_frame = pd.DataFrame({'Importance': list(importances), 'Feature': list(X_train_onehot.columns)})
importance_frame.sort_values(by = 'Importance', inplace = True)

In [ ]:
importance_frame.plot(kind = 'barh', x = 'Feature', figsize = (16,100), color = 'orange')

In [ ]:
fig.savefig("importance.png", format="PNG")